- Top 10 Paths mit anzahl versptungen (balkendiagramm)<br>
- Top 10 Bahnhöfe mit anzahl verspätungen geteilt durch anzahl fahrten (also quasi Prozentsatz der verspätungen am Bahnhof)(balkendiagramm)<br>
- Beides nochmal mit durchschnittlichen Verspätungslängen like 1

Anzahl delay/anzahl delay + anzahl on time 

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import date, time, datetime

In [2]:
#Create dataframe from CSV
df = pd.read_csv("../data/fromAPI/cleaned_hourly_all.csv")

In [3]:
# #Total number of delays 
# total_delays = df[df["arrival_delay_check"] == "delay"].shape[0]

# #Path delays
# path_delay_counts = df[df["arrival_delay_check"] == "delay"].groupby("path").size().reset_index(name='count')


# #Normalized counts
# path_delay_counts['normalized_count'] = path_delay_counts['count'] / total_delays

In [35]:
df2 = df.groupby("path", as_index=False).arrival_delay_check.count()
df3 = df[df["arrival_delay_check"] == "delay"].groupby("path", as_index=False).arrival_delay_check.count()
paths = pd.merge(df2, df3, how="left", on="path", suffixes=("", "_delay"))
paths["delay%"] = (paths["arrival_delay_check_delay"] / paths["arrival_delay_check"])*100

In [41]:
paths.sort_values("delay%", inplace=True, ascending=False)
paths[paths["arrival_delay_check"] > 6].head(100)


                                                    path  arrival_delay_check  \
19950  München Hbf|München-Pasing|Augsburg Hbf|Ulm Hb...                   20   
19723  München Hbf|Ingolstadt Hbf|Nürnberg Hbf|Würzbu...                   12   
472    Amsterdam Centraal|Utrecht Centraal|Arnhem Cen...                   14   
6375   Dresden Hbf|Dresden-Neustadt|Riesa|Leipzig Hbf...                    7   
6374   Dresden Hbf|Dresden-Neustadt|Riesa|Leipzig Hbf...                    7   
...                                                  ...                  ...   
27406  Wien Hbf|Wien Meidling|St.Pölten Hbf|Linz Hbf|...                   14   
2838   Berlin Ostbahnhof|Berlin Hbf|Berlin-Spandau|Wo...                   27   
11303  Hamburg Hbf|Hannover Hbf|Göttingen|Kassel-Wilh...                   13   
28488  Zürich HB|Basel SBB|Basel Bad Bf|Freiburg(Brei...                   13   
28487  Zürich HB|Basel SBB|Basel Bad Bf|Freiburg(Brei...                   12   

       arrival_delay_check_

In [90]:
# # Sort the normalized counts to get the top 10 paths
# top_10_normalized_counts = path_delay_counts.sort_values(by='normalized_count', ascending=False).head(10)

# #Percentage
# top_10_normalized_counts["percentage"] = top_10_normalized_counts["normalized_count"] * 100

In [91]:
#showing the 10 paths with the biggest percentage of delays
# Create a bar chart using Plotly
fig = px.bar(
    paths,
    x='path',
    y='percentage',
    title='Top 10 Paths with Highest Proportional delay amount',
    labels={'path': 'Path', 'normalized_count': 'Proportion of Delays (%)'}
)
#Update
#fig.update_xaxes(tickangle=90)
# Hide X-axis labels by setting tickvals and ticktext to empty lists
fig.update_xaxes(tickvals=[], ticktext=[])
fig.update_layout(
    height=600,
    margin=dict(l=20, r=20, t=50, b=100)
)

#actually show y in %%!
fig.update_yaxes(tickformat=".2f")

fig.show()

In [74]:
df["arrival_delay_check"].value_counts()

arrival_delay_check
on_time    1264927
delay       105662
Name: count, dtype: int64